In [1]:
import json

# JSON 파일 경로
file_path = 'C:/Fintech_service/10Final_PJT/extracted_data.json'

# 파일 불러오기 및 데이터 개수 세기
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)
    data_count = len(data)

print(f"총 데이터 개수: {data_count}개")


총 데이터 개수: 130000개


In [3]:
data

[{'bizesId': 'MA010120220800000124',
  'bizesNm': '계림찜닭',
  'brchNm': '',
  'indsLclsCd': 'I2',
  'indsLclsNm': '음식',
  'indsMclsCd': 'I201',
  'indsMclsNm': '한식',
  'indsSclsCd': 'I20110',
  'indsSclsNm': '닭/오리고기 구이/찜',
  'ksicCd': 'I56111',
  'ksicNm': '한식 일반 음식점업',
  'adongCd': '41173610',
  'rdnmCd': '411733183004',
  'rdnm': '경기도 안양시 동안구 귀인로',
  'lon': '126.956875261225',
  'lat': '37.384705286306'},
 {'bizesId': 'MA010120220800000165',
  'bizesNm': '미사리풀하우스',
  'brchNm': '',
  'indsLclsCd': 'I2',
  'indsLclsNm': '음식',
  'indsMclsCd': 'I201',
  'indsMclsNm': '한식',
  'indsSclsCd': 'I20107',
  'indsSclsNm': '돼지고기 구이/찜',
  'ksicCd': 'I56113',
  'ksicNm': '한식 육류 요리 전문점',
  'adongCd': '41450610',
  'rdnmCd': '414504406104',
  'rdnm': '경기도 하남시 미사동로40번길',
  'lon': '127.210916353843',
  'lat': '37.5611926064609'},
 {'bizesId': 'MA010120220800000192',
  'bizesNm': '빈체로',
  'brchNm': '',
  'indsLclsCd': 'I2',
  'indsLclsNm': '음식',
  'indsMclsCd': 'I204',
  'indsMclsNm': '서양식',
  'indsSclsCd

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time

# API 요청 URL 및 기본 파라미터 설정
url = "https://apis.data.go.kr/B553077/api/open/sdsc2/storeListInDong"
service_key = "VllPGBHyYqwt8xReydqLPRy08DwNkL3jvkbl/pADFjWJ28QlggjEvMQd4UpVqjxqDeV/GHBtun4WRLF+8mZjxA=="
numOfRows = 100  # 페이지당 결과 수를 100개로 설정
max_page = 8000   # 최대 페이지 수 설정 (예: 100페이지까지)

# 요청을 통해 데이터를 가져오는 함수 정의
def fetch_data(pageNo):
    params = {
        "ServiceKey": service_key,    # 인증키 (인코딩된 형태로 입력)
        "pageNo": pageNo,             # 페이지 번호
        "numOfRows": numOfRows,       # 한 페이지 결과 수
        "divId": "ctprvnCd",          # 구분 ID (시도 코드 사용)
        "key": "41",                  # 시도 코드 값
        "indsLclsCd": "I2",           # 업종 대분류 코드
        "type": "xml"                 # 응답 타입 (xml 또는 json)
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        items = root.find(".//items")
        page_data = []
        if items is not None and len(items) > 0:
            for item in items:
                data = {
                    "bizesId": item.findtext("bizesId"),
                    "bizesNm": item.findtext("bizesNm"),
                    "brchNm": item.findtext("brchNm"),
                    "indsLclsCd": item.findtext("indsLclsCd"),
                    "indsLclsNm": item.findtext("indsLclsNm"),
                    "indsMclsCd": item.findtext("indsMclsCd"),
                    "indsMclsNm": item.findtext("indsMclsNm"),
                    "indsSclsCd": item.findtext("indsSclsCd"),
                    "indsSclsNm": item.findtext("indsSclsNm"),
                    "ksicCd": item.findtext("ksicCd"),
                    "ksicNm": item.findtext("ksicNm"),
                    "adongCd": item.findtext("adongCd"),
                    "rdnmCd": item.findtext("rdnmCd"),
                    "rdnm": item.findtext("rdnm"),
                    "lon": item.findtext("lon"),
                    "lat": item.findtext("lat")
                }
                page_data.append(data)
            return page_data
        else:
            return None
    else:
        print(f"페이지 {pageNo}에서 요청 실패: {response.status_code}")
        return None

# 병렬 처리를 통해 데이터를 빠르게 수집하여 DataFrame에 저장
all_data = []
print("데이터 추출 중...")

with ThreadPoolExecutor(max_workers=10) as executor:  # 최대 10개의 스레드 사용
    futures = {executor.submit(fetch_data, page): page for page in range(1, max_page + 1)}
    for future in tqdm(as_completed(futures), total=len(futures), desc="페이지 처리 중"):
        page_data = future.result()
        if page_data:
            all_data.extend(page_data)

# DataFrame으로 변환
df = pd.DataFrame(all_data)

# 결과 출력 (또는 파일로 저장)
print(f"\n총 데이터 개수: {len(df)}")
print(df.head())  # 데이터의 상위 5개 행을 출력


데이터 추출 중...


페이지 처리 중:  41%|█████████████████████████▏                                    | 3256/8000 [11:48<17:12,  4.60it/s]


In [9]:
# DataFrame을 JSON 파일로 변환하고 저장
json_file_path = "extracted_data1000.json"
df.to_json(json_file_path, orient="records", force_ascii=False, indent=4)

print(f"데이터가 JSON 파일로 '{json_file_path}'에 저장되었습니다.")

데이터가 JSON 파일로 'extracted_data1000.json'에 저장되었습니다.
